In [2]:
from cassandra.cluster import Cluster
import time
import datetime
import random
from IPython.display import clear_output

### Connect to keyspace 'csc8101'

In [3]:
cluster = Cluster()
session = cluster.connect('csc8101')

### create table for task1
- Design of Visits_per_url
 - Query First: the query made on this table is, return pages by given (Topic,Batchtime,Client id), there is an assumption: no rows count equals 0. In this case, the order of key (Topic,Batchtime,Client id) before (page) can satisfiy the requirement. 
 - Choice of Partition Key and Cluster Key: There is a trade-off between Partition Key and Cluster Key. Partition Key will seprarte the whole tabale into partitions and each will exisit in one or more nodes, and cluster key will order data in one partition. 
   - If there are too many partition key, the hashing for partition key will have too many conflicts. So I do not use combined partiiton key e.g. (topic Batchtime).
   - If we use Batchtime as partition key, then all writing or querying in a period of time will all happened in one partition. That can not take advantage of Cassandra distributed system, which not all nodes or partitions are woring together. So I select Topic as partition keys, and others as cluster key.
- the table Visit_per_url_tmp only used in generate dummy data. Not used in spark streamming case.
 - when generating dummy data, we need to figure out distince users for a given topic and page, so the order of key should be (Topic,batchtime,page) , Clientid. To satisfy this, I duplicate the data in Visits_per_url.

In [3]:
t1_drop = '''
    Drop Table IF EXISTS Visits_per_url
'''
t1_table = 'CREATE TABLE IF NOT EXISTS Visits_per_url(\
            Batchtime timestamp,\
            Clientid varchar,\
            Topic varchar,\
            Page varchar,\
            count int,\
            PRIMARY KEY(Topic,Batchtime,Clientid,Page)\
            )'
session.execute(t1_drop)
session.execute(t1_table)
t1_tmp_drop = '''
    Drop Table IF EXISTS Visits_per_url_tmp
'''
t1_tmp_table = 'CREATE TABLE IF NOT EXISTS Visits_per_url_tmp(\
            Batchtime timestamp,\
            Clientid varchar,\
            Topic varchar,\
            Page varchar,\
            count int,\
            PRIMARY KEY(Topic,Batchtime,Page,Clientid)\
            )'
session.execute(t1_tmp_drop)
session.execute(t1_tmp_table)

### Design of table 2
- According to the query requirement, we need get ordered count by given Topic and Batchtime. In this case we need to define Topic and Batchtime as Partition key and count as cluster key for order.

In [15]:
t2_drop = '''
    Drop Table IF EXISTS Distinct_user_counts
'''
t2_table='''
            CREATE TABLE IF NOT EXISTS Distinct_user_counts(
            Batchtime timestamp,
            Topic varchar,
            Page varchar,
            count int,
            PRIMARY KEY((Topic,Batchtime),count,Page)
            ) 
        '''
#WITH CLUSTERING ORDER BY (insertion_time DESC);
session.execute(t2_drop)
session.execute(t2_table)

### User_Batchtime used for ordering batchtime
- In task 3, we need to get the most recently time and the past 3 time period. To implement this in Discinct_user_counts table, we need to use distince count which is not suggested. So I choose to build a addition table which stores clientid as parititon key and batchtime as cluster key.

In [6]:
t3_drop='''
    Drop Tabel IF EXISTS User_Batchtime
'''
t3_table='''
        CREATE TABLE IF NOT EXISTS User_Batchtime(
        Clientid varchar,
        Batchtime timestamp,
        PRIMARY KEY(Clientid,Batchtime)
        )
'''
#session.execute(t3_drop)
session.execute(t3_table)

#### helper function used for generating dummy data

In [5]:
t1_insert = '''
                INSERT INTO Visits_per_url (Topic,Batchtime,Page,Clientid,count) values (%s,%s,%s,%s,%s)
                    '''
t1_insert_tmp = '''
                INSERT INTO Visits_per_url_tmp (Topic,Batchtime,Page,Clientid,count) values (%s,%s,%s,%s,%s)
                    '''
def t1_update(time,clientid,topic,page,count):
    session.execute(t1_insert,(topic,time,page,clientid,count))
    session.execute(t1_insert_tmp,(topic,time,page,clientid,count))

    
t2_insert = '''
                    INSERT into Distinct_user_counts (Batchtime,Topic,count,page) values(%s,%s,%s,%s)
'''
def t2_update(time,topic,page,count):
    session.execute(t2_insert,(time,topic,count,page))
    
def get_milltime(t):
    return int(time.mktime(t.timetuple()) *1000)

t1_distinct_client = '''
        select count(Clientid) from Visits_per_url_tmp
        where Batchtime = %s AND
        Topic = %s AND
        Page = %s
'''
def t1_unique_client_count(time,topic,page):
    re = session.execute(t1_distinct_client,(time,topic,page))
    for row in re:
        return row[0]
    
t3_insert = '''
    INSERT into User_Batchtime(Clientid,Batchtime) values (%s,%s)
'''

def t3_update(clientid,batchtime):
    session.execute(t3_insert,(clientid,batchtime))

#### Insert dummy data


In [6]:
num_client = 200
num_topic = 10
num_page = 50
num_time = 5
verbose = 5000

#random params
page_access_rate = 0.1
page_access_rate_repeat=0.5

#first time access rate 0.8 
# if access : next access rate = 0.5

In [9]:
total = num_topic*num_page*num_time*num_client
current = 0
for nt in range(num_time):
    for ntopic in range(num_topic):
        for npage in range(num_page):
            for nclient in range(num_client):
                d = datetime.date(2017,1,1+nt)
                batchtime = get_milltime(d)
                topic = 'Topic'+str(ntopic)
                page = 'Page'+str(npage)
                clientId = 'Client'+str(nclient)
                count = 0
                rate = page_access_rate
                while (random.random()<rate):
                    count+=1
                    rate = page_access_rate_repeat
                
                if count>0:
                    t1_update(batchtime,clientId,topic,page,count)
                    t3_update(clientId,batchtime)
                if current % verbose ==0:
                    clear_output()
                    print ('%d / %d' %(current,total))
                current +=1
                

495000 / 500000


- test if insert for table 1 success

In [26]:
rows = session.execute('SELECT count(*) FROM Visits_per_url')
for row in rows:
    print (row)
rows = session.execute('SELECT count(*) FROM User_Batchtime')
for row in rows:
    print (row)

Row(count=50312)
Row(count=1049)


- insert table 2

In [16]:
verbose = 100
total = num_topic*num_page*num_time
current = 0
for nt in range(num_time):
    for ntopic in range(num_topic):
        for npage in range(num_page):
            d = datetime.date(2017,1,1+nt)
            batchtime = get_milltime(d)
            topic = 'Topic'+str(ntopic)
            page = 'Page'+str(npage)
            count = t1_unique_client_count(batchtime,topic,page)
            t2_update(batchtime,topic,page,count)
            if current % verbose ==0:
                clear_output()
                print ('%d / %d' %(current,total))
            current +=1

2400 / 2500


In [25]:
rows = session.execute('SELECT count(*) FROM Distinct_user_counts')
for row in rows:
    print (row)

Row(count=2640)


- TASK 3

In [18]:
nt = 2
ntopic = 7
nclient = 5
past = 3
topn = 10
topic = 'Topic'+str(ntopic)
clientId = 'Client'+str(nclient)

In [19]:
t2_top_n = '''
            SELECT * FROM Distinct_user_counts 
            where Batchtime = %s AND Topic = %s  
            order by count DESC limit %s
'''
def get_top_n(batchtime,topic,n):
    rows = session.execute(t2_top_n,(batchtime,topic,n))
    return rows

In [20]:
t3_get_rank_batchtime = '''
    SELECT Batchtime FROM User_batchtime 
    where Clientid = %s  
    order by Batchtime DESC limit %s
'''
def get_past_batchtime(clientId,past):
    rows = session.execute(t3_get_rank_batchtime,(clientId,past))
    return rows
t1_get_current_batchtime='''
    SELECT Batchtime From Visits_per_url
    where Topic = %s 
    order by Batchtime DESC limit 1
'''
def get_current_batchtime(topic):
    rows = session.execute(t1_get_current_batchtime,[topic])
    for row in rows:
        return row[0]
t1_get_previous = '''
        SELECT * FROM Visits_per_url
        where Topic = %s AND
        Batchtime = %s AND 
        Clientid = %s
'''
def get_previous(topic,time,clientid):
    rows = session.execute(t1_get_previous,(topic,time,clientid))
    return rows

In [21]:
current_time = get_current_batchtime(topic)
past_time=[]
rows = get_past_batchtime(clientId,past)
for row in rows:
    #print (row.batchtime)
    past_time.append(row.batchtime)
print (past_time)

[datetime.datetime(2017, 1, 5, 0, 0), datetime.datetime(2017, 1, 4, 0, 0), datetime.datetime(2017, 1, 3, 0, 0)]


In [22]:
visit_page = set()
for i in range(past):
    pasttime = past_time[i]
    pasttime = get_milltime(pasttime)
    rows = get_previous(topic,pasttime,clientId)
    for row in rows:
        visit_page.add(row.page)

In [23]:
recommend = []
rows = get_top_n(current_time,topic,topn+len(visit_page))
for row in rows:
    recommend.append(row.page)
new_recommend = [item for item in recommend if item not in visit_page]
if(len(new_recommend)>topn):
    new_recommend = new_recommend[:topn]

In [24]:
print(recommend)
print(visit_page)
print(new_recommend)

['Page33', 'Page2', 'Page21', 'Page43', 'Page14', 'Page10', 'Page37', 'Page18', 'Page11', 'Page30', 'Page49', 'Page28', 'Page23', 'Page22', 'Page1', 'Page9', 'Page24', 'Page8', 'Page48', 'Page46', 'Page34', 'Page47', 'Page41', 'Page36', 'Page25']
{'Page10', 'Page40', 'Page42', 'Page23', 'Page35', 'Page48', 'Page5', 'Page19', 'Page20', 'Page49', 'Page32', 'Page21', 'Page11', 'Page30', 'Page46'}
['Page33', 'Page2', 'Page43', 'Page14', 'Page37', 'Page18', 'Page28', 'Page22', 'Page1', 'Page9']
